# BTC 💰📈 15-min Time Frame analysis

The purpose of this notebook is to analyse Bitcoin's price in a 15-minute time frame.It analyses the price change within the past year when the RSI indicator drops below 80 or crosses above 30. It consists of the following:

1) Read data, EDA and data preprocessing

2) Calculating RSI

3) Identifying Bitcoin's up and down trends

4) Analysing the start and end date, price changes, percentage changes and number of candles in each trend


### Import libraries and Read Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import pandas_ta as pta
from datetime import datetime

In [2]:
# Read the dataset with pandas

dataset = pd.read_csv('BTC.csv')
print('Datasets shape:', dataset.shape)

Datasets shape: (350688, 2)


In [3]:
dataset.head(7)

,close,timestamp
0,4.58,1325376000
1,4.58,1325376900
2,4.58,1325377800
3,4.58,1325378700
4,4.58,1325379600
5,4.58,1325380500
6,4.58,1325381400


In [4]:
dataset.tail(7)

,close,timestamp
350681,46434.81,1640988900
350682,46396.86,1640989800
350683,46290.58,1640990700
350684,46303.98,1640991600
350685,46429.65,1640992500
350686,46350.44,1640993400
350687,46465.54,1640994300


### EDA and Data Preprocessing

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350688 entries, 0 to 350687
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   close      350688 non-null  float64
 1   timestamp  350688 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 5.4 MB


In [6]:
dataset.isnull().sum()

close        0
timestamp    0
dtype: int64

In [7]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
close,350688.0,7.884545e+03,1.418994e+04,4.140000e+00,2.704975e+02,9.124200e+02,8.605285e+03,6.871889e+04
timestamp,350688.0,1.483185e+09,9.111154e+07,1.325376e+09,1.404281e+09,1.483185e+09,1.562090e+09,1.640994e+09


In [8]:
# Convert unix time to human readable format 

start_date = datetime.fromtimestamp(dataset['timestamp'].iloc[0]).strftime("%b %d %Y %H:%M:%S")
end_date = datetime.fromtimestamp(dataset['timestamp'].iloc[-1]).strftime("%b %d %Y %H:%M:%S")

print('Start Date: ', start_date)
print('End Date: ', end_date)

Start Date:  Jan 01 2012 03:30:00
End Date:  Jan 01 2022 03:15:00


In [9]:
df = dataset.copy()
df.head(3)

,close,timestamp
0,4.58,1325376000
1,4.58,1325376900
2,4.58,1325377800


In [10]:
df['time'] = ''
times = []

for time in df['timestamp']:
    times.append(datetime.fromtimestamp(time).strftime("%b %d %Y %H:%M:%S"))

df['time'] = times
df.head(3)

,close,timestamp,time
0,4.58,1325376000,Jan 01 2012 03:30:00
1,4.58,1325376900,Jan 01 2012 03:45:00
2,4.58,1325377800,Jan 01 2012 04:00:00


In [11]:
del df['timestamp']
df.head(3)

,close,time
0,4.58,Jan 01 2012 03:30:00
1,4.58,Jan 01 2012 03:45:00
2,4.58,Jan 01 2012 04:00:00


In [12]:
# Select last year's data 

df = df.tail(35040)     # 365(Days) * 96(Candle per day)     
df = df.reset_index()   
df.head(3)

,index,close,time
0,315648,28985.51125,Jan 01 2021 03:30:00
1,315649,28773.91875,Jan 01 2021 03:45:00
2,315650,28856.53125,Jan 01 2021 04:00:00


In [13]:
del df['index']
df.head(3)

,close,time
0,28985.51125,Jan 01 2021 03:30:00
1,28773.91875,Jan 01 2021 03:45:00
2,28856.53125,Jan 01 2021 04:00:00


In [14]:
start_date = df['time'].iloc[0]
end_date = df['time'].iloc[-1]

print('Datasets shape:', df.shape)
print('Start Date: ', start_date)
print('End Date: ', end_date)

Datasets shape: (35040, 2)
Start Date:  Jan 01 2021 03:30:00
End Date:  Jan 01 2022 03:15:00


### Calculate RSI 

The Relative Strength Index (RSI), is a momentum oscillator that measures the speed and change of price movements. The RSI oscillates between zero and 100. Traditionally the RSI is considered overbought when above 70 and oversold when below 30.

In this section RSI is implemented in two ways. The first function is written using the "pandas_ta" library and the second function is written by myself.

In this notebook the first function has been used while the second function can also be used.

#### Calculate RSI using pandas_ta

In [15]:
def calculate_rsi(close_price, time_period=14):
    rsi = list(pta.rsi(close_price, time_period))
    return rsi

df['rsi'] = ''
rsi = calculate_rsi(df['close'], 14)
df['rsi'] = rsi

print('5 RSI samples: ', rsi[15:20])
df.tail(3)

5 RSI samples:  [59.836025448315866, 60.07459367805899, 60.644715691853136, 63.26140621186176, 50.40241955208979]


,close,time,rsi
35037,46429.65,Jan 01 2022 02:45:00,45.825678
35038,46350.44,Jan 01 2022 03:00:00,43.648455
35039,46465.54,Jan 01 2022 03:15:00,47.548184


#### Calculate RSI function

In [16]:
rsi_df = df.copy()
del rsi_df['time']
del rsi_df['rsi']

rsi_df.head(3)

,close
0,28985.51125
1,28773.91875
2,28856.53125


In [17]:
def rsi(close_price, time_period = 14):
    
    # Calculate price differences
    close_price['diff'] = close_price.diff(1)
    
    # Calculate avg. gains/losses
    close_price['gain'] = close_price['diff'].clip(lower=0)
    close_price['loss'] = close_price['diff'].clip(upper=0).abs()
    close_price['avg_gain'] = close_price['gain'].rolling(window=time_period, 
                                                          min_periods=time_period).mean()[:time_period+1]
    close_price['avg_loss'] = close_price['loss'].rolling(window=time_period, 
                                                          min_periods=time_period).mean()[:time_period+1]
    
    # Average gains
    for i, row in enumerate(close_price['avg_gain'].iloc[time_period+1:]):
        close_price['avg_gain'].iloc[i + time_period + 1] =\
        (close_price['avg_gain'].iloc[i + time_period] * (time_period - 1) +\
        close_price['gain'].iloc[i + time_period + 1]) / time_period
    
    # Average losses
    for i, row in enumerate(close_price['avg_loss'].iloc[time_period+1:]):
        close_price['avg_loss'].iloc[i + time_period + 1] =\
        (close_price['avg_loss'].iloc[i + time_period] * (time_period - 1) +\
        close_price['loss'].iloc[i + time_period + 1]) / time_period
    
    # Calculate RS values
    close_price['rs'] = close_price['avg_gain'] / close_price['avg_loss']

    # Calculate RSI
    close_price['rsi'] = 100 - (100 / (1.0 + rsi_df['rs']))
    
    return close_price

rsi = rsi(rsi_df, time_period = 14)
rsi

,close,diff,gain,loss,avg_gain,avg_loss,rs,rsi
0,28985.51125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28773.91875,-211.5925,0.0000,211.5925,NaN,NaN,NaN,NaN
2,28856.53125,82.6125,82.6125,0.0000,NaN,NaN,NaN,NaN
3,28960.97375,104.4425,104.4425,0.0000,NaN,NaN,NaN,NaN
4,29026.08625,65.1125,65.1125,0.0000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
35035,46290.58000,-106.2800,0.0000,106.2800,53.479055,76.747722,0.696816,41.066097
35036,46303.98000,13.4000,13.4000,0.0000,50.616265,71.265742,0.710247,41.528907
35037,46429.65000,125.6700,125.6700,0.0000,55.977247,66.175331,0.845893,45.825678
35038,46350.44000,-79.2100,0.0000,79.2100,51.978872,67.106379,0.774574,43.648455


### Calculate Break 80

This function checks all the points within the past year's RSI and returns the points that drop below 80.

In [18]:
df['break_80'] = 0
df.tail(3)

,close,time,rsi,break_80
35037,46429.65,Jan 01 2022 02:45:00,45.825678,0
35038,46350.44,Jan 01 2022 03:00:00,43.648455,0
35039,46465.54,Jan 01 2022 03:15:00,47.548184,0


In [19]:
def calculate_break_80(dataframe):
    for index in range(len(dataframe) - 1):
        first = dataframe['rsi'][index]
        second = dataframe['rsi'][index + 1]
        if first >= 80 and second < 80:
            dataframe['break_80'][index + 1] = 1
    return dataframe

break_80 = calculate_break_80(df)
df.head(3)

,close,time,rsi,break_80
0,28985.51125,Jan 01 2021 03:30:00,NaN,0
1,28773.91875,Jan 01 2021 03:45:00,NaN,0
2,28856.53125,Jan 01 2021 04:00:00,NaN,0


In [20]:
df['break_80'].value_counts()

0    34963
1       77
Name: break_80, dtype: int64

### Calculate Break 30

This function checks all the points within the past year's RSI and returns the points across 30.

In [21]:
df['break_30'] = 0
df.tail(3)

,close,time,rsi,break_80,break_30
35037,46429.65,Jan 01 2022 02:45:00,45.825678,0,0
35038,46350.44,Jan 01 2022 03:00:00,43.648455,0,0
35039,46465.54,Jan 01 2022 03:15:00,47.548184,0,0


In [22]:
def calculate_break_30(dataframe):
    for index in range(len(df) - 1):
        first = dataframe['rsi'][index]
        second = dataframe['rsi'][index + 1]
        if first <= 30 and second > 30:
            dataframe['break_30'][index + 1] = 1
    return dataframe

break_30 = calculate_break_30(df)
df.head(3)

,close,time,rsi,break_80,break_30
0,28985.51125,Jan 01 2021 03:30:00,NaN,0,0
1,28773.91875,Jan 01 2021 03:45:00,NaN,0,0
2,28856.53125,Jan 01 2021 04:00:00,NaN,0,0


In [23]:
df['break_30'].value_counts()

0    34549
1      491
Name: break_30, dtype: int64

### Downtrends

This function uses break 80 points to identify all downtrends within the past year's data. It returns start and end dates, number of candles, price variation and It's percentage for each of those downtrends.

In [24]:
downtrends = pd.DataFrame(columns=['start_date', 'end_date', 'number_of_candles', 'price_change', 'percentage'])
downtrends

,start_date,end_date,number_of_candles,price_change,percentage


In [25]:
def calculate_downtrends(input_dataframe, res_dataframe):
    # Fill start date
    for item in input_dataframe['break_80']:
        if item == 1:
            index = input_dataframe[input_dataframe['break_80'] == item].index.values
            res_dataframe['start_date'] = input_dataframe['time'][index]
            
    for date in res_dataframe['start_date']:
        res_dataframe_index = int(res_dataframe[res_dataframe['start_date'] == date].index.values)
        input_dataframe_index = int(input_dataframe[input_dataframe['time'] == date].index.values)
        rsi = input_dataframe['rsi'][input_dataframe_index]  
        i = 1
        first = rsi
        second = input_dataframe['rsi'][input_dataframe_index + i]
        if first <= second: 
            # The downtrend did not continue
            res_dataframe['end_date'][res_dataframe_index] = date
            res_dataframe['number_of_candles'][res_dataframe_index] = 0
            res_dataframe['price_change'][res_dataframe_index] = 0
            res_dataframe['percentage'][res_dataframe_index] = 0
        else:
            number_of_candles = 0
            # The downtrend continued
            while first > second:
                first = second
                i += 1
                number_of_candles += 1
                second = input_dataframe['rsi'][input_dataframe_index + i]
                index = int(input_dataframe[input_dataframe['rsi'] == first].index.values)
                res_dataframe['end_date'][res_dataframe_index] = input_dataframe['time'][index]
                res_dataframe['number_of_candles'][res_dataframe_index] = number_of_candles
                res_dataframe['price_change'][res_dataframe_index] = input_dataframe['close'][index] - \
                        input_dataframe['close'][input_dataframe_index]
                res_dataframe['percentage'][res_dataframe_index] = ((input_dataframe['close'][index] - \
                        input_dataframe['close'][input_dataframe_index]) / df['close'][input_dataframe_index]) * 100 
    return res_dataframe 

calculate_downtrends(df, downtrends)

,start_date,end_date,number_of_candles,price_change,percentage
148,Jan 02 2021 16:30:00,Jan 02 2021 16:30:00,0,0,0
153,Jan 02 2021 17:45:00,Jan 02 2021 18:00:00,1,-263.77571,-0.840612
169,Jan 02 2021 21:45:00,Jan 02 2021 21:45:00,0,0,0
224,Jan 03 2021 11:30:00,Jan 03 2021 11:45:00,1,-374.26715,-1.085615
650,Jan 07 2021 22:00:00,Jan 07 2021 22:15:00,1,-1300.3925,-3.324882
...,...,...,...,...,...
30531,Nov 15 2021 04:15:00,Nov 15 2021 04:15:00,0,0,0
31866,Nov 29 2021 02:00:00,Nov 29 2021 02:30:00,2,-286.07,-0.507974
31875,Nov 29 2021 04:15:00,Nov 29 2021 04:15:00,0,0,0
34006,Dec 21 2021 09:00:00,Dec 21 2021 09:15:00,1,-219.58,-0.452278


### Uptrends

This function uses break 30 points to identify all uptrends within the past year's data. It returns start and end dates, number of candles, price variation and It's percentage for each of those uptrends.

In [26]:
uptrends = pd.DataFrame(columns=['start_date', 'end_date', 'number_of_candles', 'price_change', 'percentage'])
uptrends

,start_date,end_date,number_of_candles,price_change,percentage


In [27]:
def calculate_uptrends(input_dataframe, res_dataframe):
    for item in input_dataframe['break_30']:
        if item == 1:
            index = input_dataframe[input_dataframe['break_30'] == item].index.values
            res_dataframe['start_date'] = input_dataframe['time'][index]
            
    for date in res_dataframe['start_date']:
        res_dataframe_index = int(res_dataframe[res_dataframe['start_date'] == date].index.values)
        input_dataframe_index = int(input_dataframe[input_dataframe['time'] == date].index.values)
        rsi = input_dataframe['rsi'][input_dataframe_index]
        i = 1
        first = rsi
        second = input_dataframe['rsi'][input_dataframe_index + i]
        if first >= second: 
            res_dataframe['end_date'][res_dataframe_index] = date
            res_dataframe['number_of_candles'][res_dataframe_index] = 0
            res_dataframe['price_change'][res_dataframe_index] = 0
            res_dataframe['percentage'][res_dataframe_index] = 0
        else:
            number_of_candles = 0
            while first < second:
                first = second
                i += 1
                number_of_candles += 1
                second = input_dataframe['rsi'][input_dataframe_index + i]
                index = int(input_dataframe[input_dataframe['rsi'] == first].index.values)
                res_dataframe['end_date'][res_dataframe_index] = input_dataframe['time'][index]
                res_dataframe['number_of_candles'][res_dataframe_index] = number_of_candles
                res_dataframe['price_change'][res_dataframe_index] = input_dataframe['close'][index] -\
                        input_dataframe['close'][input_dataframe_index]
                res_dataframe['percentage'][res_dataframe_index] = ((input_dataframe['close'][index] -\
                        input_dataframe['close'][input_dataframe_index]) /\
                        input_dataframe['close'][input_dataframe_index]) * 100 
    return res_dataframe

calculate_uptrends(df, uptrends)

,start_date,end_date,number_of_candles,price_change,percentage
181,Jan 03 2021 00:45:00,Jan 03 2021 01:00:00,1,133.89143,0.425055
332,Jan 04 2021 14:30:00,Jan 04 2021 14:30:00,0,0,0
684,Jan 08 2021 06:30:00,Jan 08 2021 07:00:00,2,1052.17,2.828732
913,Jan 10 2021 15:45:00,Jan 10 2021 16:00:00,1,136.18036,0.344517
936,Jan 10 2021 21:30:00,Jan 10 2021 21:30:00,0,0,0
...,...,...,...,...,...
34805,Dec 29 2021 16:45:00,Dec 29 2021 16:45:00,0,0,0
34807,Dec 29 2021 17:15:00,Dec 29 2021 17:30:00,1,500.17,1.066404
34848,Dec 30 2021 03:30:00,Dec 30 2021 03:30:00,0,0,0
34850,Dec 30 2021 04:00:00,Dec 30 2021 04:30:00,2,194.09,0.417407
